In [3]:
# adds scripts/ and src/ folder: so you can import scripts/functions across project steps
import sys 
sys.path.append("../../src")
sys.path.append("../../scripts")

from os import listdir
from os import path

import pandas as pd
from tqdm.notebook import tqdm

from data_filepaths import image_folders, portraits_csv, images_with_boxes_folder, object_detection_results_folder

In [4]:
object_detection_results_folder

'../../data/object_detection_results'

In [6]:
results_per_image = [
    pd.read_csv(path.join(object_detection_results_folder, csv_path))
    for csv_path in listdir(object_detection_results_folder)
]

In [7]:
# ,detection_class_labels,detection_class_entities,detection_scores,x0,y0,x1,y1,new_filename
relevant_columns = ["detection_class_labels", "detection_class_entities", "detection_scores", "new_filename"]

def unique_detection_classes(result_dtf):
    return result_dtf[relevant_columns].groupby(
        ['detection_class_labels', "detection_class_entities", "new_filename"], as_index=False).agg(
        {'detection_scores':max}
    )

In [8]:
global_uresults = pd.concat([unique_detection_classes(res) for res in results_per_image])

In [9]:
global_uresults

,detection_class_labels,detection_class_entities,new_filename,detection_scores
0,58,b'Dress',81232.jpg,0.133596
1,69,b'Person',81232.jpg,0.193191
2,97,b'Chair',81232.jpg,0.256209
3,228,b'Woman',81232.jpg,0.314604
4,334,b'Plant',81232.jpg,0.086035
...,...,...,...,...
9,391,b'Tree',67763.jpg,0.070742
10,433,b'Clothing',67763.jpg,0.381404
11,502,b'Human face',67763.jpg,0.818555
12,503,b'Human arm',67763.jpg,0.133247


In [14]:
nb_portraits_per_object_class = global_uresults.groupby(
        ['detection_class_labels', "detection_class_entities"], as_index=False).agg(
        {'detection_scores':max, "new_filename": "nunique"}
    )
nb_portraits_per_object_class["nb_portraits"] = nb_portraits_per_object_class["new_filename"]
del nb_portraits_per_object_class["new_filename"]
nb_portraits_per_object_class.sort_values(by="nb_portraits", ascending=False, inplace=True)
nb_portraits_per_object_class[0:40]

,detection_class_labels,detection_class_entities,detection_scores,nb_portraits
88,433,b'Clothing',0.677999,378
100,502,b'Human face',0.981114,376
18,69,b'Person',0.617947,355
3,15,b'Human eye',0.194702,316
67,292,b'Human head',0.168908,306
60,253,b'Human hair',0.190700,297
101,503,b'Human arm',0.289596,295
55,228,b'Woman',0.698380,266
81,391,b'Tree',0.251974,264
108,568,b'Human nose',0.185795,246
